# Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import heapq

# Functions To Be Used

In [4]:
linearList = list()

def imagePuzzle(pathImg: str, patchSize: int = 16):
    img = Image.open(pathImg)
    img = img.resize((512, 512))
    print("Original Image: ")
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img = np.asarray(img)
    imgSize = img.shape[0]

    numPatches = imgSize // patchSize

    for i, patchHeight in enumerate(range(0, imgSize, patchSize)):
        for j, patchWidth in enumerate(range(0, imgSize, patchSize)):
            image = img[patchHeight: patchHeight + patchSize, patchWidth: patchWidth + patchSize]
            linearList.append(image)
    linearList[-1] = np.zeros_like(image)
    linearList2 = linearList.copy()

    random.shuffle(linearList2)

    # Graph making
    shuffledGrid = {}
    originalGrid = {}
    i = 0
    for x in range(0, numPatches):
        for y in range(0, numPatches):
            if np.array_equiv(linearList2[i], np.zeros_like(image)):
                blank = (x, y)
            shuffledGrid[(x, y)] = linearList2[i]
            originalGrid[(x, y)] = linearList[i]
            i += 1

    return originalGrid, shuffledGrid, blank, numPatches

def plotGrid(grid: dict, numPatches=16):
    img = 0
    for i in range(0, numPatches):
        if (i, 0) in grid:
            patch = grid[i, 0]
            for j in range(1, numPatches):
                if (i, j) in grid:
                    patch = np.hstack((patch, grid[i, j]))
            if isinstance(img, int):
                img = patch
            else:
                img = np.vstack((img, patch))
    plt.imshow(img)
    plt.axis(False)
    plt.show()

def makeSimpleGridDict(grid):  # convert tuple dict to simple dict
    gridDict = {}
    key = int(0)
    while True:
        if key == 64:
            break
        for i in range(0, 8):
            for j in range(0, 8):
                singlePatch = grid[i, j]
                gridDict[key] = singlePatch
                key += 1
    return gridDict

def manhattanDistance(state):
    goalState = [i for i in range(64)]
    distance = 0
    for tile in state:
        if tile != 0:
            currentRow, currentCol = divmod(tile, 8)
            goalRow, goalCol = divmod(goalState[tile], 8)
            distance += abs(currentRow - goalRow) + abs(currentCol - goalCol)
    return distance

def swapNewValuesWithShuffled(shuffledList: list(), shuffledGrid):
    newDict = {}
    j = 0
    for i in shuffledList:
        newDict[j] = shuffledGrid[i]
        j += 1
    return newDict

def convertSimpleDictToGrid(simpleDict):
    newGrid = {}
    k = 0
    for i in range(0, 8):
        for j in range(0, 8):
            newGrid[(i, j)] = simpleDict[k]
            k += 1
    return newGrid

def getValidMoves(state):
    zero = None
    for key, value in state.items():
        if np.array_equal(value, np.zeros_like(value)):
            zero = key
            break
    validMoves = []
    r, c = zero
    if r > 0:
        validMoves.append((r - 1, c))
    if r < 7:
        validMoves.append((r + 1, c))
    if c > 0:
        validMoves.append((r, c - 1))
    if c < 7:
        validMoves.append((r, c + 1))
    moveIndexes = []
    for i in validMoves:
        index = (i[0] * 8) + i[1]
        moveIndexes.append(index)
    return moveIndexes

def getZeroIndex(state):
    zero = None
    for key, value in state.items():
        if np.array_equal(value, np.zeros_like(value)):
            zero = key
            break
    index = (zero[0] * 8) + zero[1]
    return index

def compareShuffledToOriginal(shuffledGrid, originalGridDict):
    shuffledList = []
    for key, value in shuffledGrid.items():
        for KEY in range(0, 64):
            if value.all() == originalGridDict[KEY].all():
                if KEY not in shuffledList:
                    shuffledList.append(KEY)
    return shuffledList

def aStarSearch(shuffled):
    goalState = [i for i in range(64)]
    heuristics = {}
    visitedStates = set()
    lotisg = compareShuffledToOriginal(shuffled, originalGridDict)
    heuristics[manhattanDistance(lotisg)] = lotisg
    queue = [(manhattanDistance(lotisg), lotisg)]
    counter = 0
    while queue:
        _, currentState = heapq.heappop(queue)
        if currentState == goalState:
            return "Reached"
        visitedStates.add(tuple(currentState))
        newShuffledDict = swapNewValuesWithShuffled(lotisg, makeSimpleGridDict(shuffled))
        shuffled = convertSimpleDictToGrid(newShuffledDict)
        for move in getValidMoves(shuffled):
            newState = currentState[:]
            temp = newState[getZeroIndex(shuffled)]
            newState[getZeroIndex(shuffled)] = newState[move]
            newState[move] = temp
            if tuple(newState) not in visitedStates:
                newHeuristic = manhattanDistance(newState)
                heuristics[newHeuristic] = newState
                heapq.heappush(queue, (newHeuristic, newState))
                if counter % 2 == 0:
                    plotGrid(shuffled)
        counter += 1
    return False

# Driver Code

In [ ]:
originalGrid, shuffledGrid, blank, numPatches = imagePuzzle("C:/Users/hassa/airBaloons.jpg", 64)
originalGridDict = makeSimpleGridDict(originalGrid)

aStarSearch(shuffledGrid)

---

**Note:** I have tried my best to provide accurate results in this notebook. However, these results may not be entirely accurate, and contributions or corrections are encouraged. Thank you!
